# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# @title Notebook setup.

%cd ..
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.patches as patches
from semantic_routing.benchmark import utils
from semantic_routing.benchmark.graphs import grid_graph
from semantic_routing.benchmark.graphs import city_graph

# Road Graphs

We provide two sources of road graphs: a rich simulation of a grid-based road network and real-world road graphs sampled from major US cities. We sample metadata such as points-of-interest and travel times according to either simulation (for grid graphs) or OpenStreetMap data (for US cities).

## Travel Times

On grid graphs, the estimated travel times are sampled from a Poisson distribution for residential roads or set to 1 for highway roads. Estimated travel time may differ for the same road depending on which direction one is driving.

In [2]:
# @title Estimated travel times of an example grid road graph.

poi_specs = utils.get_poi_specs()
graph = grid_graph.GridGraph(poi_specs, 64, splits=[0.95, 0, 0.05], highway_density_node=50, min_node_gap=10)

offset = 0.1
c = 1
new_graph = nx.DiGraph()
for (u, v, data) in list(graph.nx_graph.edges(data=True)):
  if u[0] == v[0] and u[1] > v[1]:
    data["dir"] = "Down Road"
    new_graph.add_edge((u[0] + offset, u[1] - offset / c), (v[0]  + offset, v[1] + offset / c), **data)
  elif u[1] == v[1] and u[0] > v[0]:
    data["dir"] = "Left Road"
    new_graph.add_edge((u[0] - offset / c, u[1]  + offset), (v[0] + offset / c, v[1] + offset), **data)
  elif u[0] == v[0] and u[1] < v[1]:
    data["dir"] = "Up Road"
    new_graph.add_edge((u[0] - offset, u[1] + offset / c), (v[0] - offset, v[1] - offset / c), **data)
  elif u[1] == v[1] and u[0] < v[0]:
    data["dir"] = "Right Road"
    new_graph.add_edge((u[0] + offset / c, u[1] - offset), (v[0] - offset / c, v[1] - offset), **data)
  else:
    data["dir"] = "Highway"
    new_graph.add_edge(u, v, **data)


plt.figure(figsize=(8, 8))
nx.draw(
    new_graph,
    {(x, y): (x, y) for x, y in new_graph.nodes()},
    edge_color="#dedede",
    node_color="none",
)

colors = {"Up Road": "blue", "Down Road": "orange", "Left Road": "green", "Right Road": "red", "Highway": "black"}
for (u, v, d) in new_graph.edges(data=True):
  color = colors[d["dir"]]
  travel_time = int(d['travel_time'])
  mid_point = [(u[0] + v[0]) / 2, (u[1] + v[1]) / 2]
  plt.text(mid_point[0], mid_point[1], s=travel_time, ha='center', va='center', color=color)

legend_labels = []
for k, v in colors.items():
  legend_labels.append(
      patches.Patch(color=v, label=k)
  )
plt.title("Example grid road graph with expected travel times.")
plt.legend(
    handles=legend_labels,
    loc="lower center",
    fontsize=10,
    ncol=len(legend_labels),
)
plt.show()

To compute live travel times, we sample a binomial random variable and scale expected travel time upwards accordingly.

In [3]:
# @title Current travel times of an example grid road graph.

offset = 0
new_graph = nx.DiGraph()
for (u, v, data) in list(graph.nx_graph.edges(data=True)):
  if u[0] == v[0] and u[1] > v[1]:
    data["dir"] = "down"
    new_graph.add_edge((u[0] + offset, u[1] - offset / c), (v[0]  + offset, v[1] + offset / c), **data)
  elif u[1] == v[1] and u[0] < v[0]:
    data["dir"] = "right"
    new_graph.add_edge((u[0] + offset / c, u[1] - offset), (v[0] - offset / c, v[1] - offset), **data)
  elif u[1] == v[1] and u[0] > v[0]:
    continue
  elif u[0] == v[0] and u[1] < v[1]:
    continue
  else:
    data["dir"] = "highway"
    new_graph.add_edge(u, v, **data)


plt.figure(figsize=(8, 8))
nx.draw(
    new_graph,
    {(x, y): (x, y) for x, y in new_graph.nodes()},
    edge_color="#dedede",
    node_color="none",
)
for (u, v, d) in new_graph.edges(data=True):
  travel_time = int(d['travel_time'])
  current_travel_time = int(d['current_travel_time'])
  mid_point = [(u[0] + v[0]) / 2, (u[1] + v[1]) / 2]
  if d["dir"] == "highway":
    mid_point[1] += 0.1
    plt.text(mid_point[0]-0.1, mid_point[1]-0.1, s=travel_time, ha='center', va='center', color="green")
    plt.text(mid_point[0]+0.1, mid_point[1]+0.1, s=current_travel_time, ha='center', va='center', color="orange")
  else:
    plt.text(mid_point[0]-0.1, mid_point[1]-0.1, s=travel_time, ha='center', va='center', color="blue")
    plt.text(mid_point[0]+0.1, mid_point[1]+0.1, s=current_travel_time, ha='center', va='center', color="red")


legend_labels = [patches.Patch(color="blue", label="Estimated Time"), patches.Patch(color="red", label="Current Time"),
                 patches.Patch(color="green", label="Estimated Time (Highway)"), patches.Patch(color="orange", label="Current Time (Highway)")]

plt.legend(
    handles=legend_labels,
    loc="lower center",
    fontsize=10,
    ncol=len(legend_labels),
)
plt.title("Example grid road graph with expected and current travel times.")
plt.show()

On US city road graphs, the estimated travel times are obtained from the publicly sourced estimates of the OpenStreetMap project, which are computed based on published speed limits, road conditions, and road lengths.

In [4]:
# @title Current travel times of a subgraph of Berkeley's road graph.

graph = city_graph.CityGraph(poi_specs, 100, seed=1, splits=[0.95, 0, 0.05], use_test_city=True)

pos = {}
ego = None
plotted = []
for u, v, data in graph.nx_graph.edges(data=True):
  if (v, u) in plotted:
    continue
  plotted.append((u, v))
  if ego is None:
    ego = data["u_lat"], data["u_lon"]
  if u not in pos:
    pos[u] = np.array([2000 * (data["u_lat"] - ego[0]), 2000 * (data["u_lon"] - ego[1])])
  if v not in pos:
    pos[v] = np.array([2000 * (data["v_lat"] - ego[0]), 2000 * (data["v_lon"] - ego[1])])

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plt.axis("off")
for ax in axes:
  nx.draw(graph.nx_graph, pos, ax=ax, node_size=0, arrows=False,
    edge_color="#dedede",
    node_color="none",)

rng = random.Random(0)
for (u, v, d) in graph.nx_graph.edges(data=True):
  if rng.random() < 0.9:
    continue
  travel_time = int(10 * d['travel_time']) / 10
  current_travel_time = int(10 * d['current_travel_time']) / 10
  highway = d['highway']
  u = pos[u]
  v = pos[v]
  mid_point = [(u[0] + v[0]) / 2, (u[1] + v[1]) / 2]
  axes[0].text(mid_point[0], mid_point[1], s=current_travel_time, ha='center', va='center', color="green")
  axes[1].text(mid_point[0], mid_point[1], s=travel_time, ha='center', va='center', color="blue")
  axes[2].text(mid_point[0], mid_point[1], s=highway, ha='center', va='center', color="red")

legend_labels = [patches.Patch(color="blue", label="Estimated Time"), patches.Patch(color="green", label="Current Time"), patches.Patch(color="red", label="Highway")]
plt.legend(
    handles=legend_labels,
    loc="lower center",
    fontsize=10,
    ncol=len(legend_labels),
)

plt.title("Example road graph from Berkeley with expected and current travel times.")
plt.show()

## Working with road graphs

Each road graph can be perceived as a subgraph of a giant graph, that is taken around some "central node". We form these subgraphs, for example, from US city road networks by picking a random central node and taking a k-hop neighborhood around it.
The edges around the central node can be accessed through the property `graph.central_edges`.

To sample non-central edges, we can call `graph.sample_noncentral_edge(split, rng)`.
The set of non-central edges is divided into different data splits, which you choose when calling `sample_noncentral_edge` (0 is training, 1 is validation, 2 is testing).

In [5]:
# @title Example of a road graph's central and noncentral edges.
num_nodes = 20000 # @param
graph = city_graph.CityGraph(poi_specs, num_nodes, seed=0, splits=[0.95, 0, 0.05], use_test_city=True)
rng = random.Random(0)

pos = {}
ego = None
for u, v, data in graph.nx_graph.edges(data=True):
  if ego is None:
    ego = data["u_lat"], data["u_lon"]
  if u not in pos:
    pos[u] = np.array([1000 * (data["u_lat"] - ego[0]), 1000 * (data["u_lon"] - ego[1])])
  if v not in pos:
    pos[v] = np.array([1000 * (data["v_lat"] - ego[0]), 1000 * (data["v_lon"] - ego[1])])

plt.figure(figsize=(12, 12))
plt.axis("off")

random_edges = []
for _ in range(100):
  random_edges.append(graph.sample_noncentral_edge(split=2, rng=rng))
colors = []
widths = []
for u, v in graph.nx_graph.edges():
  if graph.edge_from_internal[(u, v)] in random_edges:
    colors.append('red')
    widths.append(10)
  elif graph.edge_from_internal[(u, v)] in graph.central_edges:
    colors.append('green')
    widths.append(5)
  else:
    colors.append("#dedede")
    widths.append(1)

nx.draw(graph.nx_graph, pos, node_size=0, arrows=False,
    edge_color=colors,
    node_color="none",
        width=widths
        )

legend_labels = [patches.Patch(color="green", label="Central Edge"),
                 patches.Patch(color="#dedede", label="Noncentral Edge"),
                 patches.Patch(color="red", label="Randomly Sampled Noncentral Edges")]
plt.legend(
    handles=legend_labels,
    loc="lower center",
    fontsize=10,
    ncol=len(legend_labels),
)
plt.axis("off")
plt.title("Example road graph from Berkeley with central edges highlighted.")
plt.show()

We can use `get_edge_pois` to iterate through the POIs located on an edge and use `get_road_type` to get the road type of the edge.

In [6]:
# @title Examples of get_edge_pois and get_road_type.

poi_specs = utils.get_poi_specs()
general_poi_specs, specialized_poi_specs = poi_specs

poi_type_to_id = {}
id_to_poi_type = {}
general_pois, specialized_pois = poi_specs
for poi_info in general_pois + sum(specialized_pois.values(), []):
  id_to_poi_type[poi_info['poi_type_id']] = poi_info['poi_type_name']
  poi_type_to_id[poi_info['poi_type_name']] = poi_info['poi_type_id']

rng = random.Random(0)
for _ in range(100):
  edge = graph.sample_noncentral_edge(split=2, rng=rng)
  if len(graph.get_edge_pois(edge)) > 0:
    break
print("Calling `get_edge_pois` on edge {}:".format(edge))
for i, (node_id, poi_type) in enumerate(graph.get_edge_pois(edge)):
  print("POI {}: {} (ID #{})".format(i, " and ".join([id_to_poi_type[p] for p in poi_type]), node_id))
print()
print("Calling `get_road_type` on edge {}: {}".format(edge, graph.get_road_type(edge)))

Road graphs provide two ways of retrieving the neighborhood around an edge. The first way is calling `get_reachable(edge)`, which returns edges that are outgoing from the end of `edge`.

In [13]:
# @title Visualization of reachable edges.

graph = city_graph.CityGraph(poi_specs, 8, seed=0, splits=[0.95, 0, 0.05], use_test_city=True)

edge = graph.central_edges[-1]
reachable = graph.get_reachable(edge)
print("Edges reachable from {}: {}".format(edge, reachable))

pos = {}
ego = None
c = 10000
for i, (u, v, data) in enumerate(graph.nx_graph.edges(data=True)):
  if ego is None:
    ego = c * data["u_lat"], c * data["u_lon"]
  if u not in pos:
    pos[u] = np.array([c * data["u_lat"] - ego[0], c * data["u_lon"] - ego[1]], dtype=np.float64)
  if v not in pos:
    pos[v] = np.array([c * data["v_lat"] - ego[0], c* data["v_lon"] - ego[1]], dtype=np.float64)

plt.figure(figsize=(7, 7))
plt.axis("off")
rng = random.Random(0)

edge_colors = []
width = []
for (u, v, d) in graph.nx_graph.edges(data=True):
  new_edge = graph.edge_from_internal[(u, v)]
  u = pos[u]
  v = pos[v]
  mid_point = [(u[0] + v[0]) / 2, (u[1] + v[1]) / 2]
  offset = 0.3 * (rng.random() - 0.5)
  if new_edge in reachable:
    edge_colors.append("green")
    plt.text(mid_point[0], mid_point[1] + offset, s=new_edge, ha='center', va='center', color="green")
    width.append(3)
  elif new_edge == edge:
    edge_colors.append("red")
    plt.text(mid_point[0], mid_point[1] + offset, s="Edge {}".format(edge), ha='center', va='center', color="red")
    width.append(3)
  else:
    edge_colors.append("none")
    width.append(1)

nx.draw(graph.nx_graph, pos, arrows=False,
    edge_color=edge_colors,
        alpha=0.3,
    width=width,
    node_color="black", node_size=2)

legend_labels = [patches.Patch(color="green", label="Reachable Edge"),
                 patches.Patch(color="red", label="Central Edge")]
plt.legend(
    handles=legend_labels,
    loc="lower center",
    fontsize=10,
    ncol=len(legend_labels),
)
plt.axis("off")
plt.title("Edges reachable from edge {}".format(edge))
plt.show()

Another way is to use `get_receptive_field` to get a wider neighborhood around a node.

In [8]:
# @title Receptive field visualization.

receptive_field_size = 256 # @param
num_nodes = 20000 # @param

graph = city_graph.CityGraph(poi_specs, num_nodes, seed=0, splits=[0.95, 0, 0.05], use_test_city=True)

receptive_field = graph.get_receptive_field(graph.central_edges[0], receptive_field_size)
receptive_field = [p[0] for p in receptive_field]
for edge in receptive_field.copy():
  data = graph.nx_graph.get_edge_data(*graph.edge_to_internal[edge])
  receptive_field += [graph.edge_from_internal[e] for e in data["edges"]]
  assert len(data["edges"]) == data["level"]
receptive_field = set(receptive_field)
pos = {}
ego = None
c = 10000
for i, (u, v, data) in enumerate(graph.nx_graph.edges(data=True)):
  if ego is None:
    ego = c * data["u_lat"], c * data["u_lon"]
  if u not in pos:
    pos[u] = np.array([c * data["u_lat"] - ego[0], c * data["u_lon"] - ego[1]], dtype=np.float64)
  if v not in pos:
    pos[v] = np.array([c * data["v_lat"] - ego[0], c* data["v_lon"] - ego[1]], dtype=np.float64)

plt.figure(figsize=(12, 12))
plt.axis("off")
rng = random.Random(0)

edge_colors = []
width = []
for (u, v, d) in graph.nx_graph.edges(data=True):
  new_edge = graph.edge_from_internal[(u, v)]
  if new_edge in receptive_field:
    edge_colors.append("green")
    width.append(1)
  elif new_edge == graph.central_edges[0]:
    edge_colors.append("red")
    width.append(5)
  else:
    edge_colors.append("#dedede")
    width.append(0.4)

nx.draw(graph.nx_graph, pos, arrows=False,
    edge_color=edge_colors,
    width=width, node_size=0)

legend_labels = [patches.Patch(color="green", label="Receptive Field Edge"),
                 patches.Patch(color="#dedede", label="Other Edges"),
                 patches.Patch(color="red", label="Central Edge")]
plt.legend(
    handles=legend_labels,
    loc="lower center",
    fontsize=10,
    ncol=len(legend_labels),
)
plt.axis("off")
plt.title("Receptive field around edge {}".format(edge))
plt.show()

`get_receptive_field` also indicates, for each edge it returns, what the edge you should take next to reach the given edge, based on the usual (but not necessarily current) travel times.

In [9]:
# @title Receptive field edge hints.

receptive_field_size = 100 # @param

graph = city_graph.CityGraph(poi_specs, 100, seed=0, splits=[0.95, 0, 0.05], use_test_city=True)

edge = graph.central_edges[0]
receptive_field = graph.get_receptive_field(edge, receptive_field_size)
receptive_edge, receptive_cand = receptive_field[-1]
print("Receptive field size: {}".format(edge, len(receptive_field)))

pos = {}
ego = None
c = 10000
for i, (u, v, data) in enumerate(graph.nx_graph.edges(data=True)):
  if ego is None:
    ego = c * data["u_lat"], c * data["u_lon"]
  if u not in pos:
    pos[u] = np.array([c * data["u_lat"] - ego[0], c * data["u_lon"] - ego[1]], dtype=np.float64)
  if v not in pos:
    pos[v] = np.array([c * data["v_lat"] - ego[0], c* data["v_lon"] - ego[1]], dtype=np.float64)

plt.figure(figsize=(12, 12))
plt.axis("off")
rng = random.Random(0)
print(receptive_cand)

edge_colors = []
width = []
for (u, v, d) in graph.nx_graph.edges(data=True):
  new_edge = graph.edge_from_internal[(u, v)]
  u = pos[u]
  v = pos[v]
  mid_point = [(u[0] + v[0]) / 2, (u[1] + v[1]) / 2]
  offset = 2 * (rng.random() - 0.5)
  if new_edge == receptive_edge:
    edge_colors.append("green")
    width.append(3)
  elif new_edge == edge:
    edge_colors.append("red")
    width.append(3)
  elif new_edge == receptive_cand:
    edge_colors.append("blue")
    width.append(5)
  else:
    edge_colors.append("#dedede")
    width.append(1)

nx.draw(graph.nx_graph, pos, arrows=False,
    edge_color=edge_colors,
    width=width, node_size=0)

legend_labels = [patches.Patch(color="green", label="Receptive Field Edge"),
                 patches.Patch(color="#dedede", label="Other Edges"),
                 patches.Patch(color="blue", label="Edge to Reach Receptive Field Edge"),
                 patches.Patch(color="red", label="Central Edge")]
plt.legend(
    handles=legend_labels,
    loc="lower center",
    fontsize=10,
    ncol=len(legend_labels),
)
plt.axis("off")
plt.title("Sample from receptive field around edge {}".format(edge))
plt.show()

## Routes

Road graphs provide utilities for directly computing optimal
We can use `get_shortest_path_len` to get the shortest path between two edges. We can pass along information, such as preferring/avoiding highways and point-of-interest requests to the function.

In [10]:
# @title Shortest path examples.
graph = city_graph.CityGraph(poi_specs, 20000, seed=0, splits=[0.95, 0, 0.05], use_test_city=True)

rng = random.Random(0)
edge = graph.central_edges[0]
other_edge = graph.sample_noncentral_edge(2, rng)
shortest_dist, shortest_path = graph.get_shortest_path_len(edge, other_edge, return_path=True)
shortest_dist_pref, shortest_path_pref = graph.get_shortest_path_len(edge, other_edge, {"linear": "dislike highways", "pois": ((129,),)}, return_path=True)
shortest_dist_poi, shortest_path_poi = graph.get_shortest_path_len(edge, other_edge, {"linear": "", "pois": ((129,),)}, return_path=True)
shortest_dist_poi_or, shortest_path_poi_or = graph.get_shortest_path_len(edge, other_edge, {"linear": "", "pois": ((130, 129),)}, return_path=True)
shortest_dist_poi_and, shortest_path_poi_and = graph.get_shortest_path_len(edge, other_edge, {"linear": "", "pois": ((129,), (130,))}, return_path=True)
print("Shortest path time: {}".format(shortest_dist))
print("Shortest path segment length: {}".format(len(shortest_path)))
print("Shortest path time that visits a computer shop: {}".format(shortest_dist_poi))
print("Shortest path segment length that visits a computer shop: {}".format(len(shortest_path_poi)))
print("Shortest path time that visits a computer shop and minimizes highway time: {}".format(shortest_dist_pref))
print("Shortest path segment length that visits a computer shop and minimizes highway time: {}".format(len(shortest_path_pref)))
print("Shortest path time that visits a computer shop or electronics shop: {}".format(shortest_dist_poi_or))
print("Shortest path segment length that visits a computer shop or electronics shop: {}".format(len(shortest_path_poi_or)))
print("Shortest path time that visits a computer shop and electronics shop: {}".format(shortest_dist_poi_and))
print("Shortest path segment length that visits a computer shop and electronics shop: {}".format(len(shortest_path_poi_and)))

pos = {}
ego = None
c = 10000
for i, (u, v, data) in enumerate(graph.nx_graph.edges(data=True)):
  if ego is None:
    ego = c * data["u_lat"], c * data["u_lon"]
  if u not in pos:
    pos[u] = np.array([c * data["u_lat"] - ego[0], c * data["u_lon"] - ego[1]], dtype=np.float64)
  if v not in pos:
    pos[v] = np.array([c * data["v_lat"] - ego[0], c* data["v_lon"] - ego[1]], dtype=np.float64)


rng = random.Random(0)

edge_color_lists = [
    [],
    [],
    [],
    [],
    []
]
width_lists = [
    [],
    [],
    [],
    [],
    []
]
for (u, v, d) in graph.nx_graph.edges(data=True):
  new_edge = graph.edge_from_internal[(u, v)]
  u = pos[u]
  v = pos[v]
  mid_point = [(u[0] + v[0]) / 2, (u[1] + v[1]) / 2]
  offset = 2 * (rng.random() - 0.5)
  for edge_color_list, width_list, path in zip(edge_color_lists, width_lists, [shortest_path, shortest_path_poi, shortest_path_pref, shortest_path_poi_or, shortest_path_poi_and]):
    found = False
    for edge in path:
      data = graph.nx_graph.get_edge_data(*graph.edge_to_internal[edge])
      if new_edge in [graph.edge_from_internal[e] for e in data["edges"]]:
        found = True
        width_list.append(3)
        if data["level"] > 1:
          edge_color_list.append("green")
        else:
          edge_color_list.append("blue")
        break
    if not found:
      edge_color_list.append("#dedede")
      width_list.append(0.2)


node_colors = []
for node in graph.nx_graph.nodes():
  if node == graph.edge_to_internal[edge][0] or node == graph.edge_to_internal[other_edge][1]:
    node_colors.append("red")
    width.append(100)
  else:
    node_colors.append("none")

fig, axes = plt.subplots(1, 3, figsize=(24, 10))
plt.axis("off")
for ax, edge_colors, width_list, title in zip(axes, edge_color_lists[:3], width_lists[:3], ["Shortest Travel", "Stop By Computer Shop", "Stop By Computer Shop; Minimize Highway Time"]):
  nx.draw(graph.nx_graph, pos, arrows=False,
      edge_color=edge_colors, ax=ax,
      width=width_list, node_size=3,
          node_color=node_colors)
  ax.set_title(title)

legend_labels = [patches.Patch(color="green", label="Returned Path"),
                 patches.Patch(color="red", label="Start/End Edge"),
]
plt.legend(
    handles=legend_labels,
    loc="lower center",
    fontsize=10,
    ncol=len(legend_labels),
)
plt.axis("off")
plt.show()

fig, axes = plt.subplots(1, 2, figsize=(16, 10))
plt.axis("off")
for ax, edge_colors, width_list, title in zip(axes, edge_color_lists[3:], width_lists[3:], ["Stop By Computer Shop or Electronics Shop", "Stop By Computer Shop and Electronics Shop"]):
  nx.draw(graph.nx_graph, pos, arrows=False,
      edge_color=edge_colors, ax=ax,
      width=width_list, node_size=3,
          node_color=node_colors)
  ax.set_title(title)

legend_labels = [patches.Patch(color="blue", label="Returned Path"),
                 patches.Patch(color="red", label="Start/End Edge"),
]
plt.legend(
    handles=legend_labels,
    loc="lower center",
    fontsize=10,
    ncol=len(legend_labels),
)
plt.axis("off")
plt.show()

Road graphs also provide statistics about a route through the method `route_metrics`.

In [11]:
# @title Examples of route metrics from a road graph.

rng = random.Random(0)
graph = city_graph.CityGraph(poi_specs, 20000, seed=0, splits=[0.95, 0, 0.05], use_test_city=True)
start = graph.central_edges[0]
end = graph.sample_noncentral_edge(0, rng)

df = []
_, path = graph.get_shortest_path_len(start, end, {"linear": "dislike highways", "pois": ((129,),)}, return_path=True)
df.append({"Path": "Visit computer shop, dislike highways", "Evaluated on": "Visit computer shop, dislike highways", **graph.route_metrics({"linear": "dislike highways", "pois": ((129,),)}, end, path)})
df.append({"Path": "Visit computer shop, dislike highways", "Evaluated on": "Visit computer shop", **graph.route_metrics({"linear": "", "pois": ((129,),)}, end, path)})
_, path = graph.get_shortest_path_len(start, end, {"linear": "", "pois": ((129,),)}, return_path=True)
df.append({"Path": "Visit computer shop", "Evaluated on": "Visit computer shop, dislike highways", **graph.route_metrics({"linear": "dislike highways", "pois": ((129,),)}, end, path)})
df.append({"Path": "Visit computer shop", "Evaluated on": "Visit computer shop", **graph.route_metrics({"linear": "", "pois": ((129,),)}, end, path)})
df.append({"Path": "Visit computer shop", "Evaluated on": "Visit computer shop and electronics shop", **graph.route_metrics({"linear": "", "pois": ((129,), (130,))}, end, path)})
df.append({"Path": "Visit computer shop", "Evaluated on": "Visit computer shop or electronics shop", **graph.route_metrics({"linear": "", "pois": ((129, 130),)}, end, path)})
_, path = graph.get_shortest_path_len(start, end, {"linear": "", "pois": ((129,), (130,))}, return_path=True)
df.append({"Path": "Visit computer shop and electronics shop", "Evaluated on": "Visit computer shop and electronics shop", **graph.route_metrics({"linear": "", "pois": ((129,), (130,))}, end, path)})
_, path = graph.get_shortest_path_len(start, end, {"linear": "", "pois": ((129,),)}, return_path=True)
path = path[:-1]
df.append({"Path": "Visit computer shop, last edge omitted", "Evaluated on": "Visit computer shop", **graph.route_metrics({"linear": "", "pois": ((129,),)}, end, path)})
pd.DataFrame(df)